In [1]:
import os
import numpy  as np
import pandas as pd
from datetime import datetime

# 경고 처리
import warnings
warnings.filterwarnings('ignore')

### 수도권 지하철 좌표
https://data.seoul.go.kr/dataList/OA-21232/S/1/datasetView.do

In [20]:
import pandas as pd
from datetime import datetime

# 수도권 지하철 위경도 데이터
df1 = pd.read_csv("C:/Users/duPar/Downloads/서울시 역사마스터 정보 (1).csv",encoding='CP949')
df1.columns = ['station_cd','station_nm','line_nm','lat','lon']
df1['base_ym'] = datetime.today().strftime("%Y%m") # 데이터 기준년월 컬럼 추가
df1.to_csv('C:\oasis\git\data\수도권 지하철 역좌표\서울시 역사마스터 정보_최종(1).csv',index=False,encoding='utf-8-sig')

In [21]:
df1

,station_cd,station_nm,line_nm,lat,lon,base_ym
0,9996,미사,5호선,127.193877,37.560927,202210
1,9995,강일,5호선,127.175930,37.557490,202210
2,4929,김포공항,김포골드라인,126.801868,37.562360,202210
3,4928,고촌,김포골드라인,126.770345,37.601243,202210
4,4927,풍무,김포골드라인,126.732387,37.612488,202210
...,...,...,...,...,...,...
759,154,종로5가,1호선,127.001849,37.570926,202210
760,153,종로3가,1호선,126.991847,37.570406,202210
761,152,종각,1호선,126.982923,37.570161,202210
762,151,시청,1호선,126.977088,37.565715,202210


### 수도권 외 지하철 좌표
https://data.kric.go.kr/rips/M_01_01/detail.do?id=32

In [268]:
# 역좌표
df = pd.read_excel('c:/Users/duPar/Downloads/전체_도시철도역사정보_20221007.xlsx')
df = df[['역번호', '역사명', '노선명', '역위도', '역경도']]

# 소숫점 너무 긴 좌표는 DW에 안 들어감
df['역위도'] = df['역위도'].astype(float)
df['역경도'] = df['역경도'].astype(float)

for i in df.index:
    df['역위도'][i] = round(df['역위도'][i],6)
    df['역경도'][i] = round(df['역경도'][i],6)

df.columns = ['station_cd', 'station_nm', 'line_nm', 'lon', 'lat']
df['base_ym'] = datetime.today().strftime("%Y%m")

# 티머니 승하차수 호선명 가져오기
DF = pd.read_csv('c:/Users/duPar/Downloads/subway_xy_202210121839.csv')
DF = DF.drop_duplicates()

# 티머니 승하차수에 있는 수도권역 삭제
for i in DF.index:
    idx = df[df['line_nm'].str.contains(DF['line_nm'][i])].index
    df = df.drop(idx,axis=0)
    
df.to_csv('C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\수도권 외 승하차수\수도권_외_도시철도역사정보_최종.csv',index=False,encoding='utf-8-sig')

In [579]:
# 경위도 데이터 채우기
for i in DF.index:
    
    idx = df[df['station_nm'].str.contains(DF['역명'][i])].index
    
    if len(idx) > 0:
        df['lat'][idx[0]] = DF['Latitude'][i]
        df['lon'][idx[0]] = DF['Longitude'][i]

최종 (대구 좌표 수정본)

In [592]:
df = pd.read_excel('C:\\Users\duPar\Downloads\좌표변환_대구_진행중.xlsx')

# 호선 데이터 채우기
df['station_cd'] = df['station_cd'].astype(str)
df['line_nm'] = np.nan

for i in df.index:
    
    if df['station_cd'][i][0] == '1':
        df['line_nm'][i] = '1호선'
    elif df['station_cd'][i][0] == '2':
        df['line_nm'][i] = '2호선'
    elif df['station_cd'][i][0] == '3':
        df['line_nm'][i] = '3호선'
    elif df['station_cd'][i][0] == '4':
        df['line_nm'][i] = '4호선'
        
df.to_csv('C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\대구지하철_좌표수정본.csv',index=False,encoding='utf-8-sig')

### 수도권 외 지하철 승하차수
1. 대전교통공사_시간대별 승하차인원 : [https://www.data.go.kr/data/15060591/fileData.do](https://www.data.go.kr/data/15060591/fileData.do)
2. 대구교통공사_역별일별시간별승하차인원현황 : [https://www.data.go.kr/data/15002503/fileData.do](https://www.data.go.kr/data/15002503/fileData.do)
3. 부산교통공사_시간대별 승하차인원 : [https://www.data.go.kr/data/3057229/fileData.do](https://www.data.go.kr/data/3057229/fileData.do)
4. 광주광역시도시철도공사_역일시간대별 승하차량 : [https://www.data.go.kr/data/15060048/fileData.do](https://www.data.go.kr/data/15060048/fileData.do)

###### 부산 승하차수

In [ ]:
# 파일 불러오기
df1 = pd.read_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\부산교통공사_시간대별 승하차인원_20200731.csv",encoding='CP949')

# 데이터 타입 변경
df1['역번호'] = df1['역번호'].astype(str)
df1['년월일'] = df1['년월일'] + datetime.today().strftime(" %H:%M:%S")

# 역 기준, 승하차수 합 구하기
df1['sum'] = df1[['01시-02시', '02시-03시', '03시-04시',
       '04시-05시', '05시-06시', '06시-07시', '07시-08시', '08시-09시', '09시-10시',
       '10시-11시', '11시-12시', '12시-13시', '13시-14시', '14시-15시', '15시-16시',
       '16시-17시', '17시-18시', '18시-19시', '19시-20시', '20시-21시', '21시-22시',
       '22시-23시', '23시-24시', '24시-01시']].groupby(df1.index).sum().sum(axis=1)
df1 = df1.drop(columns=['01시-02시', '02시-03시', '03시-04시',
       '04시-05시', '05시-06시', '06시-07시', '07시-08시', '08시-09시', '09시-10시',
       '10시-11시', '11시-12시', '12시-13시', '13시-14시', '14시-15시', '15시-16시',
       '16시-17시', '17시-18시', '18시-19시', '19시-20시', '20시-21시', '21시-22시',
       '22시-23시', '23시-24시', '24시-01시'])

# 승하차 행 > 열로 변환
df1['ride_pasgr'] = np.nan # 승차수
df1['alight_pasgr'] = np.nan # 하차수

for i in df1.index:
    
    if df1['구분'][i]=='승차':
        df1['ride_pasgr'][i] = df1['sum'][i]
        
    elif df1['구분'][i]=='하차':
        df1['alight_pasgr'][i-1] = df1['sum'][i]

# 컬럼 정리
df1 = df1.drop_duplicates('역명',keep='first').reset_index(drop=True)
df1 = df1.drop(columns=['구분','합계','sum'])
df1['base_ym'] = datetime.today().strftime("%Y%m")
df1.columns = ['station_cd', 'station_nm', 'work_datetime', 'ride_pasgr', 'alight_pasgr','base_ym']
df1.to_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\부산교통공사_시간대별 승하차인원_20200731_최종.csv",index=False,encoding='utf-8-sig')

In [496]:
### 추가 수정 (역코드 포맷, 호선명, DB INSERT)

# 역좌표 데이터 중, 호선 정보만 매칭하면 됨
df = pd.read_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\수도권_외_도시철도역사정보_최종.csv")
df = df[df['line_nm'].str.contains('부산')]
df = df.drop_duplicates('station_nm')

# 부산 호선/역코드 수정
DF = pd.read_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\부산교통공사_시간대별 승하차인원_20200731_최종.csv",encoding='utf-8')

# 역코드 포맷 좌표와 매칭 가능하게 수정
DF['station_cd'] = DF['station_cd'].astype(str)
DF['station_cd'] = ('00' + DF['station_cd']).str[-4:]

# 역좌표 데이터와 이름 통일
DF['line_nm'] = np.nan

for i in DF.index:
    
    idx = df[df['station_cd'].str.contains(DF['station_cd'][i])].index
    
    if len(idx) > 0:
        DF['station_nm'][i] = df['station_nm'][idx[0]]
        DF['line_nm'][i] = df['line_nm'][idx[0]]

# 호선명 수정
idx = df[df['line_nm'].str.contains('도시철도',na=False)].index

for i in idx:
    df['line_nm'][i] = df[df['line_nm'].str.contains('도시철도',na=False)]['line_nm'][i][-3:]
    
for i in DF.index:
    
    idx_sub = DF[DF['line_nm'].isnull()].index
    idx = df[df['station_cd'].str.contains(DF['station_cd'][i])].index
    
    if len(idx) > 0:
        DF['station_nm'][i] = df['station_nm'][idx[0]]
        DF['line_nm'][i] = df['line_nm'][idx[0]]
        

    if len(idx_sub) > 0:
        if DF['station_nm'][i][0] == '1':
            DF['line_nm'][idx_sub[0]] = '1호선'
        elif DF['station_nm'][i][0] == '2':
            DF['line_nm'][idx_sub[0]] = '2호선'
        elif DF['station_nm'][i][0] == '3':
            DF['line_nm'][idx_sub[0]] = '3호선'
        elif DF['station_nm'][i][0] == '4':
            DF['line_nm'][idx_sub[0]] = '4호선'
            

### 3. DB INSERT
import psycopg2
import datetime

df = DF
table = 'm1.subway'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-13 16:28:55.007682  :  0
112


###### 대구 승하차수 (일별, 역별)

In [220]:
file = "C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\대구교통공사_역별일별시간별승하차인원현황_20220831.csv"
df1 = pd.read_csv(file,encoding='CP949')

# 데이터 타입 변경
df1 = df1[['역번호', '역명', '승하차', '일계']]

# 역별 승하차 합
getOn  = df1[df1['승하차']=='승차'].groupby('역명').sum().reset_index()
getOff = df1[df1['승하차']=='하차'].groupby('역명').sum().reset_index()
df1 = df1.drop_duplicates('역명',keep='last').reset_index(drop=True)

# 승하차 행 > 열로 변환
df1['ride_pasgr'] = np.nan # 승차수
df1['alight_pasgr'] = np.nan # 하차수

for i in df1.index:
    df1['ride_pasgr'][i] = getOn['일계'][i]
    df1['alight_pasgr'][i] = getOff['일계'][i]
        
# 컬럼 정리
df1['일계'] = df1['일계'].astype(str)
df1['역번호'] = df1['역번호'].astype(str)
df1['work_datetime'] = datetime.today().strftime("%Y-%m-%d %H:%M:%S")
import datetime
df1['base_ym'] = datetime.datetime.fromtimestamp(os.path.getctime(file)).strftime('%Y%m')

df1 = df1[['역번호', '역명', 'work_datetime', 'base_ym','ride_pasgr', 'alight_pasgr']]
df1.columns = ['station_cd', 'station_nm', 'work_datetime', 'base_ym', 'ride_pasgr', 'alight_pasgr']

# 호선 데이터 채우기
df1['line_nm'] = np.nan

for i in df1.index:
    
    if df1['station_cd'][i][0] == '1':
        df1['line_nm'][i] = '1호선'
    elif df1['station_cd'][i][0] == '2':
        df1['line_nm'][i] = '2호선'
    elif df1['station_cd'][i][0] == '3':
        df1['line_nm'][i] = '3호선'
    elif df1['station_cd'][i][0] == '4':
        df1['line_nm'][i] = '4호선'
        
df1.to_csv('C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\대구교통공사_역별일별시간별승하차인원현황_20220831_최종.csv',index=False,encoding='utf-8-sig')

In [364]:
#### DB INSERT 수정본
DF = pd.read_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\대구교통공사_역별일별시간별승하차인원현황_20220831_최종.csv",encoding='utf-8')

### 3. DB INSERT
import psycopg2
import datetime

df = DF
table = 'm1.subway'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-13 15:30:50.116672  :  0
91


###### 대전 승하차수

In [347]:
# 대전
df1 = pd.read_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\대전교통공사_시간대별 승하차인원_20220831.csv",encoding='CP949')

# 데이터 타입 변경
df1['역번호'] = df1['역번호'].astype(str)
df1['날짜'] = df1['날짜'] + datetime.today().strftime(" %H:%M:%S")

# 역 기준, 승하차수 합 구하기
df1['sum'] = df1[['03-04시', '04-05시', '05-06시', '06-07시',
       '07-08시', '08-09시', '09-10시', '10-11시', '11-12시', '12-13시', '13-14시',
       '14-15시', '15-16시', '16-17시', '17-18시', '18-19시', '19-20시', '20-21시',
       '21-22시', '22-23시', '23-00시', '00-01시', '01-02시', '02-03시']].groupby(df1.index).sum().sum(axis=1)
df1 = df1.drop(columns=['03-04시', '04-05시', '05-06시', '06-07시',
       '07-08시', '08-09시', '09-10시', '10-11시', '11-12시', '12-13시', '13-14시',
       '14-15시', '15-16시', '16-17시', '17-18시', '18-19시', '19-20시', '20-21시',
       '21-22시', '22-23시', '23-00시', '00-01시', '01-02시', '02-03시'])

# 승하차 행 > 열로 변환
df1['ride_pasgr'] = np.nan # 승차수
df1['alight_pasgr'] = np.nan # 하차수

for i in df1.index:
    
    if df1['구분'][i]=='승차':
        df1['ride_pasgr'][i] = df1['sum'][i]
        
    elif df1['구분'][i]=='하차':
        df1['alight_pasgr'][i-1] = df1['sum'][i]
        
# 컬럼 정리
df1 = df1.drop_duplicates('역명',keep='first').reset_index(drop=True)
df1 = df1.drop(columns=['구분','sum'])
df1['base_ym'] = datetime.today().strftime("%Y%m")
df1.columns = ['work_datetime', 'station_cd', 'station_nm', 'ride_pasgr', 'alight_pasgr','base_ym']

# 호선 데이터 채우기
df1['line_nm'] = np.nan

for i in df1.index:
    
    if df1['station_cd'][i][0] == '1':
        df1['line_nm'][i] = '1호선'
    elif df1['station_cd'][i][0] == '2':
        df1['line_nm'][i] = '2호선'
    elif df1['station_cd'][i][0] == '3':
        df1['line_nm'][i] = '3호선'
    elif df1['station_cd'][i][0] == '4':
        df1['line_nm'][i] = '4호선'
        
df1.to_csv('C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\대전교통공사_시간대별 승하차인원_20220831_최종.csv',index=False,encoding='utf-8-sig')

In [350]:
#### DB INSERT 수정본
DF = pd.read_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\대전교통공사_시간대별 승하차인원_20220831_최종.csv",encoding='utf-8')

### 3. DB INSERT
import psycopg2
import datetime

df = DF
table = 'm1.subway'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-13 15:21:09.596884  :  0
22


###### 광주 승하차수

In [196]:
# 파일 불러오기
df1 = pd.read_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\수도권 외 승하차수\광주광역시도시철도공사_역일시간대별 승하차량_20220831.csv",encoding='utf-8')

# 데이터 타입 변경
df1['역번호'] = df1['역번호'].astype(str)
df1['년월일'] = df1['년월일'] + datetime.today().strftime(" %H:%M:%S")

# 역 기준, 승하차수 합 구하기
df1['sum'] = df1[['05_06', '06_07', '07_08', '08_09', '09_10','10_11', '11_12', '12_13', '13_14', '14_15', '15_16', '16_17', '17_18','18_19', '19_20', '20_21', '21_22', '22_23', '23_24']].groupby(df1.index).sum().sum(axis=1)
df1 = df1.drop(columns=['05_06', '06_07', '07_08', '08_09', '09_10','10_11', '11_12', '12_13', '13_14', '14_15', '15_16', '16_17', '17_18','18_19', '19_20', '20_21', '21_22', '22_23', '23_24'])

# 승하차 행 > 열로 변환
df1['ride_pasgr'] = np.nan # 승차수
df1['alight_pasgr'] = np.nan # 하차수

for i in df1.index:
    
    if df1['구분'][i]=='승차':
        df1['ride_pasgr'][i] = df1['sum'][i]
        
    elif df1['구분'][i]=='하차':
        df1['alight_pasgr'][i-1] = df1['sum'][i]
        
# 컬럼 정리
df1 = df1.drop_duplicates('역명',keep='first').reset_index(drop=True)
df1 = df1.drop(columns=['구분','sum'])
df1['base_ym'] = datetime.today().strftime("%Y%m")
df1.columns = ['work_datetime', 'station_cd', 'station_nm', 'ride_pasgr', 'alight_pasgr','base_ym']

# 호선 데이터 채우기
df1['line_nm'] = np.nan

for i in df1.index:
    
    if df1['station_cd'][i][0] == '1':
        df1['line_nm'][i] = '1호선'
    elif df1['station_cd'][i][0] == '2':
        df1['line_nm'][i] = '2호선'
    elif df1['station_cd'][i][0] == '3':
        df1['line_nm'][i] = '3호선'
    elif df1['station_cd'][i][0] == '4':
        df1['line_nm'][i] = '4호선'
        
df1.to_csv('C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\광주광역시도시철도공사_역일시간대별 승하차량_20220831_최종.csv',index=False,encoding='utf-8-sig')

In [354]:
#### DB INSERT 수정본
DF = pd.read_csv("C:\oasis\git\data\전국 지하철, 버스 (역좌표, 승하차수)\전국 지하철 좌표, 승하차수\광주광역시도시철도공사_역일시간대별 승하차량_20220831_최종.csv",encoding='utf-8')

### 3. DB INSERT
import psycopg2
import datetime

df = DF
table = 'm1.subway'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-13 15:23:07.702595  :  0
41


### 수도권 버스 좌표
http://data.seoul.go.kr/dataList/OA-15067/S/1/datasetView.do

In [14]:
df = pd.read_csv('c:/Users/duPar/Downloads/서울시버스정류소좌표데이터(2022.08.24).csv',encoding='CP949')
df.columns = ['node_id','ars_id','station_nm','x_axis','y_axis']
df['base_ym'] = datetime.today().strftime("%Y%m")
df.to_csv('c:/Users/duPar/Downloads/서울시버스정류소좌표데이터(2022.08.24)_최종.csv',index=False,encoding='utf-8-sig')

In [24]:
df

,node_id,ars_id,station_nm,x_axis,y_axis,base_ym
0,100000001,1001,종로2가사거리,126.987786,37.569764,202210
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179,202210
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709,202210
3,100000004,1004,종로2가.삼일교,126.987507,37.568582,202210
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230,202210
...,...,...,...,...,...,...
11162,124000334,25995,우성아파트,127.139326,37.550442,202210
11163,124000333,25996,우성아파트,127.140052,37.550620,202210
11164,124000332,25997,조일약국,127.123477,37.533686,202210
11165,124000331,25998,성내시장,127.125410,37.536199,202210


### 전국 버스
https://www.data.go.kr/data/15106249/fileData.do

In [25]:
df = pd.read_csv('c:/Users/duPar/Downloads/한국교통안전공단_버스정류장정보_20220830.csv',encoding='CP949')
df.columns = ['node_id','station_nm','lat','lon','local_cd','local_nm']
df['base_ym'] = datetime.today().strftime("%Y%m")
df.to_csv('c:/Users/duPar/Downloads/한국교통안전공단_버스정류장정보_20220830_최종.csv',index=False,encoding='utf-8-sig')

In [37]:
df

,node_id,station_nm,lat,lon,local_cd,local_nm,base_ym
0,150000001,대명시장건너,128.577808,35.858633,150,중구,202210
1,150000002,계명대학교대명캠퍼스건너,128.580084,35.857623,150,중구,202210
2,150000003,경북공고건너,128.582639,35.858668,150,중구,202210
3,150000004,대구남산초등학교건너,128.582410,35.862802,150,중구,202210
4,150000007,대신센트럴자이,128.577993,35.863760,150,중구,202210
...,...,...,...,...,...,...,...
46338,417000100,마산대학종점-중리방면,128.503546,35.261731,417,창원시마산회원구,202210
46339,417000102,마산시외버스터미널,128.583629,35.239006,417,창원시마산회원구,202210
46340,417000103,농산물도매시장,128.513177,35.255001,417,창원시마산회원구,202210
46341,425000547,토산삼거리,128.871498,37.396636,425,NaN,202210


###### point 변환

In [1]:
# 임포트
import numpy  as np
import pandas as pd
import psycopg2
import datetime

# 파일 불러오기
df = pd.read_csv("C:\oasis\git\data\업무 완료\전국 지하철, 버스 (역좌표, 승하차수)\한국교통안전공단_전국_버스정류장_좌표\한국교통안전공단_버스정류장정보_20220830_찐최종.csv")

# 경위도 str으로 타입 변경
df['lat'] = df['lat'].astype(str)
df['lon'] = df['lon'].astype(str)

# 경위도 포인트로 묶기
df['point'] = np.nan
for i in df.index:
    df['point'][i] = 'POINT (' + df['lat'][i] + ' ' + df['lon'][i] + ')'
    
# polygon str 변환
df['point'] = df['point'].astype(str)

C:\Users\duPar\AppData\Local\Temp\ipykernel_12944\2596277861.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'][i] = 'POINT (' + df['lat'][i] + ' ' + df['lon'][i] + ')'
C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
df = df[['base_ym', 'local_nm', 'station_nm', 'point']]

In [25]:
idx = df[df['station_nm']=='토산삼거리'].index
for i in idx:
    df['local_nm'][i] = '삼척시'

C:\Users\duPar\AppData\Local\Temp\ipykernel_12944\3973381420.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['local_nm'][i] = '삼척시'
C:\Users\duPar\Anaconda3\envs\oasis\lib\site-packages\IPython\core\interactiveshell.py:3398: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
### 2. DB 연동
table = 'm1.cremao_bus' # 테이블 지정

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    # drop & create
    cursor.execute(
    """
    drop table if exists m1.cremao_bus;
    create table m1.cremao_bus (
    base_ym        varchar(6)
    ,local_nm        varchar(50)
    ,station_nm        varchar(200)
    ,point        geography
    )
    distkey(station_nm)
    compound sortkey(base_ym, station_nm);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(%s,%s,%s,ST_GeomFromText(%s, 4326))'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-17 12:18:17.126687  :  0
2022-10-17 12:18:19.588637  :  10000
2022-10-17 12:18:22.643223  :  20000
2022-10-17 12:18:25.124606  :  30000
2022-10-17 12:18:27.221824  :  40000
46343


### 참고

In [ ]:
cur_path = os.getcwd() # 경로 설정

subway_crd = pd.read_csv('지하철역_좌표.csv', encoding='cp949')
subway_line= pd.read_csv('서울교통공사 노선별 지하철역 정보.csv', encoding='cp949')

좌표 정보는 subway_crd로, 노선 별 정보는 subway_line

In [ ]:
line_ls = []
for i in range(len(subway_crd)): 
    cur_subway = subway_crd.iloc[i]['역이름']
    matched_line = subway_line.loc[subway_line['전철역명'] == cur_subway]
    
    if len(matched_line) > 0 : 
        line_ls.append(list(matched_line['호선']))
        
    else: 
        line_ls.append(None)

subway_crd['호선'] = line_ls # 호선 정보 부여
subway_crd.dropna(subset=['호선'], inplace=True, axis=0) # 수도권 지하철역 좌표는 노선 정보가 안들어갔으므로 제거 

subway_crd['geometry'] = subway_crd.apply(lambda x: Point(x['x'], x['y']), axis=1) # 좌표 정보를 합쳐서 geometry 정보 부여  
subway_crd_gdf = GeoDataFrame(subway_crd, crs='EPSG:4326', geometry='geometry') # 좌표계 지정

In [ ]:
def write_data(data, name):
    with open(name + '.bin', 'wb') as f:
        pickle.dump(data, f)
        
        
def load_data(name):
    with open(name + '.bin', 'rb') as f:
        data = pickle.load(f)
    return data  

In [ ]:
# bin 데이터 -- 파이썬 활용 용도 (ArcGIS/QGIS 시각화 용도 x)
write_data(subway_crd_gdf, 'subway_crd_4326')
subway_crd_5179 = subway_crd_gdf.to_crs(epsg=5179) # 거리 계산을 위한 좌표계로 변환 후 저장
write_data(subway_crd_5179, 'subway_crd_5179')


# 나중에 bin 데이터 다시 불러올 때 
subway_crd_4326 = load_data('subway_crd_4326')
subway_crd_5179 = load_data('subway_crd_5179')


# 노선 정보를 빼고 좌표 데이터 
subway_crd_5179 = subway_crd_5179.drop(['호선'], axis=1)
subway_crd_5179.to_file('subway_crd_5179.geojson', driver='GeoJSON')

### 역정보, 좌표 조인

In [155]:
# 수도권 지하철 역정보
df = pd.read_csv('c:/Users/duPar/Downloads/서울교통공사 노선별 지하철역 정보_(역명,역코드)_2022.10.11.csv', encoding='CP949')
df = df.drop_duplicates('전철역명').reset_index(drop=True)
df = df[['전철역명']]
df.columns = ['지하철역명']

In [156]:
df

,지하철역명
0,신답
1,학여울
2,청량리
3,원덕
4,홍대입구
...,...
633,시청
634,종각
635,종로5가
636,제기동


In [66]:
# 수도권 지하철 수 : 638개
len(df)

638

In [111]:
# 수도권 지하철 좌표
df2 = pd.read_csv('c:/Users/duPar/Downloads/서울시 지하철역 엘리베이터 위치정보_(좌표,역명,역코드)_2022.10.11.csv', encoding='CP949')
df2 = df2.drop_duplicates('지하철역명').reset_index(drop=True)
df2 = df2[['노드 WKT','읍면동코드','지하철역명']]

In [112]:
len(df2)

272

In [128]:
# 수도권 지하철 비상콜폰 좌표
df1 = pd.read_excel('c:/Users/duPar/Downloads/인천교통공사1호선 비상콜폰.xlsx')
df1 = df1.drop_duplicates('역명').reset_index(drop=True)
df1 = df1[['선명','역명','상세위치']]

In [131]:
df1.to_excel('c:/Users/duPar/Downloads/인천교통공사1호선 비상콜폰(1).xlsx',index=False)

In [130]:
len(df1)

29

In [146]:
# 역위치 원본 데이터 병합
df1 = pd.read_csv('c:/Users/duPar/Downloads/국가철도공단_수도권1호선_역위치_20211118.csv', encoding='utf-8') # CP949
df1 = df1.drop_duplicates('역명').reset_index(drop=True)
# df1 = df1[['역명','도로명주소','지번주소']]

In [149]:
from glob import glob

In [ ]:
glob.glob('')

In [143]:
# 
df1 = pd.read_csv('c:/Users/duPar/Downloads/국가철도공단_경의중앙선_주소데이터_20211123.csv', encoding='CP949')
df1 = df1.drop_duplicates('역명').reset_index(drop=True)
# df1 = df1[['역명','도로명주소','지번주소']]

In [144]:
df1

,철도운영기관명,선명,역명,지번주소,도로명주소
0,코레일,경의중앙,임진강,경기도 파주시 문산읍 마정리 1253-3,경기도 파주시 문산읍 임진각로 115
1,코레일,경의중앙,문산,경기도 파주시 문산읍 문산리 17-14,경기도 파주시 문산읍 문산역로 94
2,코레일,경의중앙,파주(두원대학),경기도 파주시 파주읍 봉암리 487-4,경기도 파주시 파주읍 주라위길 38
3,코레일,경의중앙,월롱(서영대학교),경기도 파주시 월롱면 위전리 168-3,경기도 파주시 월롱면 통일로 980
4,코레일,경의중앙,금촌,경기도 파주시 금촌동 329-355,경기도 파주시 새꽃로 193
5,코레일,경의중앙,금릉,경기도 파주시 금촌동 605-1,경기도 파주시 금릉역로 85
6,코레일,경의중앙,운정,경기도 파주시 교하읍 야당리 295번지,경기도 파주시 운정역길 368
7,코레일,경의중앙,야당,경기도 파주시 야당동 1088,경기도 파주시 소리천로 10
8,코레일,경의중앙,탄현,경기도 고양시 일산서구 덕이동 238번지,경기도 고양시 일산서구 경의로 856
9,코레일,경의중앙,일산,경기도 고양시 일산서구 일산동 655-691,경기도 고양시 일산서구 경의로 672


In [113]:
# 좌표 데이터 병합
DF = pd.merge(df1,df2,how='right',on='지하철역명')
DF = DF[['노드 WKT_y','읍면동코드_y','지하철역명']]
DF['읍면동코드_y'] = DF['읍면동코드_y'].astype(str)

In [114]:
len(DF)

272

In [116]:
# 일치하는 역명 매칭
# 컬럼 생성
df['좌표'] = np.nan
df['법정동코드'] = np.nan

# 역명 일치하는 인덱스 칮기
for i in DF.index:
    index = df[df['지하철역명']==DF['지하철역명'][i]].index
    
    # 좌표, 법정동코드 데이터 추가
    df['좌표'][index] = DF.loc[i]['노드 WKT_y']
    df['법정동코드'][index] = DF.loc[i]['읍면동코드_y']

C:\Users\duPar\AppData\Local\Temp\ipykernel_19868\2636624923.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['좌표'][index] = DF.loc[i]['노드 WKT_y']
C:\Users\duPar\AppData\Local\Temp\ipykernel_19868\2636624923.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['법정동코드'][index] = DF.loc[i]['읍면동코드_y']


In [117]:
# 데이터 기준년월 컬럼 추가
from datetime import datetime
df['base_ym'] = datetime.today().strftime("%Y-%m-%d")

In [120]:
df

,지하철역명,좌표,법정동코드,base_ym
0,신답,NaN,NaN,2022-10-11
1,학여울,POINT(127.07085555801159 37.49648060445571),1168010600,2022-10-11
2,청량리,NaN,NaN,2022-10-11
3,원덕,NaN,NaN,2022-10-11
4,홍대입구,POINT(126.9241862213677 37.556915907782646),1144012100,2022-10-11
...,...,...,...,...
633,시청,POINT(126.97740035863328 37.566138192623036),1114010300,2022-10-11
634,종각,POINT(126.98404302021133 37.570384160886874),1111013800,2022-10-11
635,종로5가,NaN,NaN,2022-10-11
636,제기동,POINT(127.03376046869968 37.57837228457723),1123010300,2022-10-11
